In [1]:
import pandas as pd
import numpy as np
import sys
import os
import json

CODIGO PARA VER O PERFECT MICRO LEVEL, GUARDAR FICHEIRO DATASET SEM KNN, COM ZEROS PARA CALCULAR O BASE LEVEL

In [2]:
DATA_DIR = '/Users/tomas/Documents/FEUP/Tese/data/ml-20m/processed_70_10_20'

In [38]:
results = pd.read_csv(os.path.join(DATA_DIR, 'total_results_k_20.csv'))

In [39]:
results.tail()

,uid,als_ndcg,bpr_ndcg,lmf_ndcg,most_popular_ndcg,knn_cos_ndcg,knn_tfidf_ndcg,knn_bm25_ndcg
129752,129752.0,0.000000,0.000000,0.0,0.000000,0.239812,0.262650,0.000000
129753,129753.0,0.372325,0.235409,0.0,0.633159,0.629931,0.604798,0.000000
129754,129754.0,0.000000,0.386853,0.0,0.293793,0.000000,0.000000,0.122697
129755,129755.0,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,0.000000
129756,129756.0,0.437790,0.506759,0.0,0.509484,0.507093,0.515806,0.000000


In [40]:
with open(os.path.join(DATA_DIR, 'map_user.json')) as f:
    map_user = json.load(f)
    map_user_dict = {'original_id': list(map_user.keys()), 'mapped_id':list(map_user.values()) }
    map_user_df = pd.DataFrame(map_user_dict)

In [41]:
results_metadataset = pd.merge(left=map_user_df, right=results, left_on='mapped_id', right_on='uid')
results_metadataset['original_id'] = results_metadataset['original_id'].astype(np.int64)
results_metadataset = results_metadataset.drop('mapped_id', 1)
results_metadataset = results_metadataset.drop('uid', 1)
results_metadataset = results_metadataset.drop('knn_cos_ndcg', 1)
results_metadataset = results_metadataset.drop('knn_tfidf_ndcg', 1)
results_metadataset = results_metadataset.drop('knn_bm25_ndcg', 1)
results_metadataset['zeroes'] = 0

In [47]:
results_metadataset.head()

results_metadataset['zeroes'] = 0.00000000001
unique, counts = np.unique(results_metadataset.drop('original_id', 1).idxmax(axis=1), return_counts=True)
results_metadataset.drop('original_id', 1).idxmax(axis=1)
print(np.asarray((unique, counts)).T)



[['als_ndcg' 35779]
 ['bpr_ndcg' 25905]
 ['lmf_ndcg' 18691]
 ['most_popular_ndcg' 28105]
 ['zeroes' 21277]]


In [49]:
data = {'first_place':  results_metadataset.drop('original_id', 1).idxmax(axis=1)}
df = pd.DataFrame (data, columns = ['first_place'])
names = ['als_ndcg','most_popular_ndcg','bpr_ndcg','zeroes','lmf_ndcg']
total = df.shape[0] 
for name, count in zip(names,df.first_place.value_counts()):
    print("count % ",count)
    print(name," % ",count/total)

count %  35779
als_ndcg  %  0.2757384958036946
count %  28105
most_popular_ndcg  %  0.21659717780158297
count %  25905
bpr_ndcg  %  0.19964240850204612
count %  21277
zeroes  %  0.16397573926647502
count %  18691
lmf_ndcg  %  0.14404617862620128


In [32]:
results_metadataset.to_csv(os.path.join(DATA_DIR, 'results_metadataset.csv'), index=False)

In [33]:
resultados = results_metadataset.drop('original_id', 1)
resultados['zeroes'] = 0.00000000001

In [34]:
no_zeros_users = False
if no_zeros_users:
    resultados['max_value'] = resultados.max(axis=1)

In [35]:
resultados.head()

,als_ndcg,bpr_ndcg,lmf_ndcg,most_popular_ndcg,zeroes
0,0.000000,0.000000,0.0,0.000000,1.000000e-11
1,0.746604,0.000000,0.0,0.373071,1.000000e-11
2,0.000000,0.247014,0.0,0.333333,1.000000e-11
3,0.000000,0.000000,0.0,0.000000,1.000000e-11
4,0.272511,0.235409,0.0,0.371962,1.000000e-11


In [37]:
unique, counts = np.unique(resultados.idxmax(axis=1), return_counts=True)

print(np.asarray((unique, counts)).T)


[['als_ndcg' 35779]
 ['bpr_ndcg' 25905]
 ['lmf_ndcg' 18691]
 ['most_popular_ndcg' 28105]
 ['zeroes' 21277]]


In [14]:
resultados['new_col'] = resultados.stack().groupby(level=0).apply(lambda x: x.unique().tolist())

In [16]:
resultados.head(9)

,als_ndcg,bpr_ndcg,lmf_ndcg,most_popular_ndcg,zeroes,new_col
0,0.000000,0.000000,0.000000,0.000000,0,[0.0]
1,0.746604,0.000000,0.000000,0.373071,0,"[0.7466041555227104, 0.0, 0.3730707174295609]"
2,0.000000,0.247014,0.000000,0.333333,0,"[0.0, 0.2470138740984764, 0.3333333333333333]"
3,0.000000,0.000000,0.000000,0.000000,0,[0.0]
4,0.272511,0.235409,0.000000,0.371962,0,"[0.2725114255638517, 0.23540891336663824, 0.0,..."
5,0.282356,0.493568,0.304121,0.000000,0,"[0.28235577324654626, 0.4935684703397398, 0.30..."
6,0.297991,0.000000,0.000000,0.242704,0,"[0.2979914341852634, 0.0, 0.24270445668331914]"
7,0.387166,0.630930,0.377979,0.000000,0,"[0.387166384227576, 0.6309297535714575, 0.3779..."
8,0.301030,0.317575,0.000000,0.235409,0,"[0.3010299956639812, 0.317575066379576, 0.0, 0..."


In [12]:
#compute perfect
print('perfect micro: ',np.mean(resultados.max(axis=1)))
print('only als_ndcg: ', resultados['als_ndcg'].mean())
print('only bpr_ndcg: ', resultados['bpr_ndcg'].mean())
print('only lmf_ndcg: ', resultados['lmf_ndcg'].mean())
print('only most_popular_ndcg: ', resultados['most_popular_ndcg'].mean())
print('only zeroes: ', resultados['zeroes'].mean())

perfect micro:  0.46053154461827517
only als_ndcg:  0.2682470786900254
only bpr_ndcg:  0.21120017216892276
only lmf_ndcg:  0.169798831355265
only most_popular_ndcg:  0.21486503038827084
only zeroes:  0.0


0.2682470786900254

In [86]:
predictions = ['als_ndcg','zeroes', 'most_popular_ndcg','most_popular_ndcg','bpr_ndcg']

In [87]:
users = [1,2,3,4,5]

In [112]:
base_level_eval(users,predictions)

(0.11374844933999431, 0.18836259282590648, 0.26306928044453637)

In [8]:
def base_level_eval(users, predictions):
    """Uses the predctions to return the average of the ndcg impact at base level. 
    Args:
        users: list of users ids  
        predictions:predictions for users. PREDS HAVE TO be the same index ahas the users list
    Returns:
        average of ndcg
    """
    print('starting base_level_eval')
    results_algo = pd.read_csv(os.path.join(DATA_DIR, 'results_metadataset.csv'))
    base_impact = []
    base_impact_zeroes_most = []
    base_impact_zeroes_best = [] 
    for user_uid, pred in zip(users, predictions):
        
        val = results_algo.loc[ results_algo['original_id'] == user_uid, pred]
        
        if len(val.values) > 1:
            raise Exception("More than one case")
            
        if pred == 'zeroes':
            val_zeroes = results_algo.loc[ results_algo['original_id'] == user_uid, 'most_popular_ndcg']
            best = results_algo.loc[ results_algo['original_id'] == user_uid]
            base_impact.append(val.values[0])
            base_impact_zeroes_most.append(val_zeroes.values[0])
            base_impact_zeroes_best.append(best.drop('original_id', 1).max(axis=1).values[0])
        else:
            base_impact.append(val.values[0])
            base_impact_zeroes_most.append(val.values[0])
            base_impact_zeroes_best.append(val.values[0])

        

    mean = np.mean(base_impact) 
    mean_replace_zeroes_most = np.mean(base_impact_zeroes_most) 
    mean_replace_zeroes_best = np.mean(base_impact_zeroes_best)
    
    return mean, mean_replace_zeroes_most, mean_replace_zeroes_best

In [9]:
# Just choose the name of the dataset directory
DATA_DIR = '/Users/tomas/Documents/FEUP/Tese/data/ml-20m/processed_70_10_20'
is_vae = True
if is_vae:
    PARSE_DATA_DIR = os.path.join(DATA_DIR, 'embeddings/vae')
else:
    PARSE_DATA_DIR = os.path.join(DATA_DIR, 'embeddings/cdae')
file = 'metadataset_k_20.csv'
metadataset = pd.read_csv(os.path.join(PARSE_DATA_DIR, file ))

In [10]:
results_algo = pd.read_csv(os.path.join(DATA_DIR, 'results_metadataset.csv'))

In [11]:

base_level_eval(metadataset['original_id'].values,metadataset['first_place'].values)

starting base_level_eval


(0.4605315446182753, 0.4605315446182753, 0.4605315446182753)